In [ ]:
#### Training based on features of audio

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sknn.mlp import Classifier, Layer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix






#Constant
EMOTION_ANNOTATORS = {'anger': 0, 'happiness' : 1, 'sadness' : 2, 'neutral' : 3, 'frustration' : 4, 'excited': 5,
           'fear' : 6,'surprise' : 7,'disgust' : 8, 'other' : 9}

EMOTION = {'ang': 0, 'hap' : 1, 'sad' : 2, 'neu' : 3, 'fru' : 4, 'exc': 5,
           'fea' : 6,'sur' : 7,'dis' : 8, 'oth' : 9, 'xxx':10}

METHOD = {'audio_feature':0, 'LSTM':1}

#Method for classification
method = METHOD['audio_feature']

#If data is processed and saved into files, just reload, dont need to re-process
isRawDataProcessed = True

#Development mode. Only run with small data.
dev = False



def training(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    
    kf = KFold(n_splits=10, random_state=None, shuffle=False)
    i_fold = 0
    accuracy_train_results = []
    accuracy_valid_results = []

    for train_index, valid_index in kf.split(X_train):
        i_fold = i_fold + 1
        
        x_train_sub, x_valid_sub = X_train[train_index], X_train[valid_index]
        y_train_sub, y_valid_sub = y_train[train_index], y_train[valid_index]
        clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(20,), random_state=1)
        clf.fit(x_train_sub, y_train_sub)
        
        score = clf.score(x_train_sub, y_train_sub)
        score1 = clf.score(x_valid_sub, y_valid_sub)
        accuracy_train_results.append(score)
        accuracy_valid_results.append(score1)
        
        print("Score of training set: ", score)
        print("Score of validation set: ", score1)
     
       
    
    avg_accuracy_train_result = np.sum(accuracy_train_results) / len(accuracy_train_results)
    avg_accuracy_valid_result = np.sum(accuracy_valid_results) / len(accuracy_valid_results)
    print("Average accuracy training set, std:", avg_accuracy_train_result, " ",\
          np.std(accuracy_train_results))
    print("Average accuracy validation set, std:", avg_accuracy_valid_result," ", \
          np.std(accuracy_valid_results))     
    
    clf.fit(X_train, y_train)
   

    predicts = clf.predict(X_test)
    pro = clf.predict_proba(X_test)
 #   print("predicts: ", predicts)
 #   print("prob: ", pro[0])
    score_test = clf.score(X_test, y_test)
    print("\nScore for test set: ", score_test)
    print ("\nConfusion matrix:..................... ")
    matrix = confusion_matrix(y_test, predicts)
    matrix_ratio = matrix/matrix.sum(1, keepdims=True)
    print(matrix)
    print("\n", "Confusion matrix ratio:")
    print(matrix_ratio)
    print("\n", "Horizontal of confusion matrix ratio:")
    hor = [matrix_ratio[i,i] for i in range(0, len(matrix_ratio))]
    print(hor)
 

if (method == METHOD['audio_feature']):

    ##Loading  data from files
    filehandlerInput = open('processed-data/input.obj', 'rb')
    filehandlerOutput = open('processed-data/output.obj', 'rb')
    input = pickle.load(filehandlerInput)
    output = pickle.load(filehandlerOutput)

    # Get quantiry of each label
    y = np.bincount(output)
    ii = np.nonzero(y)[0]
    a = list(zip(ii,y[ii]))
    print("EMOTION_ANNOTATE: ", EMOTION_ANNOTATORS)
    print("\nThe quantity of each label: ", a, "\n")

    #Remove labels that have small quantity.
    indices = [] 
    for i in range(0, len(output)):
        if output[i] >= 6:
            indices.append(i)
    input = np.delete(input, indices, axis = 0)
    output = np.delete(output, indices)

    #Training and testing
    training(input, output)




